In [33]:
import pandas as pd

prefix = 'python'
# prefix = 'r'

In [34]:
dataAnswers = pd.read_csv('data/{}/Answers.csv'.format(prefix), encoding='latin-1')
dataQuestions = pd.read_csv('data/{}/Questions.csv'.format(prefix), encoding='latin-1')
dataTags = pd.read_csv('data/{}/Tags.csv'.format(prefix))

In [35]:
dataAnswers.info()

min(dataAnswers['CreationDate']), max(dataAnswers['CreationDate'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250788 entries, 0 to 250787
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Id                250788 non-null  int64  
 1   OwnerUserId       250000 non-null  float64
 2   CreationDate      250788 non-null  object 
 3   ParentId          250788 non-null  int64  
 4   Score             250788 non-null  int64  
 5   IsAcceptedAnswer  250788 non-null  bool   
 6   Body              250788 non-null  object 
dtypes: bool(1), float64(1), int64(3), object(2)
memory usage: 11.7+ MB


('2008-09-17T03:43:22Z', '2017-09-24T23:55:22Z')

In [36]:
dataQuestions.info()

min(dataQuestions['CreationDate']), max(dataQuestions['CreationDate'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189930 entries, 0 to 189929
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Id            189930 non-null  int64  
 1   OwnerUserId   188729 non-null  float64
 2   CreationDate  189930 non-null  object 
 3   Score         189930 non-null  int64  
 4   Title         189930 non-null  object 
 5   Body          189930 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 8.7+ MB


('2008-09-16T21:40:29Z', '2017-09-24T23:57:51Z')

In [37]:
dataQuestions['query_column'] = dataQuestions['Body'].map(lambda val: val.count('sklearn')>0)

In [38]:
result = dataQuestions.loc[dataQuestions['query_column'] == True]
len(result)

78